## Import Packages

In [1]:
import pandas as pd
import numpy as np
import pdfquery
import json
from shapely.geometry import box
from shapely.ops import cascaded_union
from math import floor
import pickle
from pymongo import MongoClient
from os import listdir
from os.path import isfile, join, isdir
from os import listdir
import re
import pickle
import hashlib
import unidecode
from fuzzysearch import find_near_matches
import textract
import unicodedata
import pydblite
import warnings

## Read Files

In [2]:
# KeywordDir="Keyword Creation"
KeywordDir = 'Keyword_Creation'
keyword_files= [join(KeywordDir, f) for f in listdir(KeywordDir)\
              if isfile(join(KeywordDir, f)) and f[-4:].lower() == "xlsx"]

In [3]:
keyword_files

['Keyword_Creation/N10 - REQUIREMENTS.xlsx',
 'Keyword_Creation/N1 - CLAIMS ADMITTED.xlsx',
 'Keyword_Creation/N3 - ENFORCEMENT ORDERS.xlsx',
 'Keyword_Creation/N27 - REQUERIMIENTO CONCURSAL.xlsx',
 'Keyword_Creation/N34 - CASE SUSPENDED ON LACK OF LEGAL GROUNDS.xlsx',
 'Keyword_Creation/N7 - HEARINGS.xlsx',
 'Keyword_Creation/N14 - CANCELLED HEARING.xlsx',
 'Keyword_Creation/N5 - REJECTED CLAIMS.xlsx',
 'Keyword_Creation/N24 - FILED.xlsx',
 'Keyword_Creation/N36 - SUSPENSION LIFTED.xlsx',
 'Keyword_Creation/N2 - TRANSFER OF LEGAL REPRESENTATION PERMITTED.msg.xlsx',
 'Keyword_Creation/N16 -  PENDING ASSET INQUIRY.xlsx',
 'Keyword_Creation/N19 - FURTHER GARNISHMENT.xlsx',
 'Keyword_Creation/N35 - PROCEDURE SUSPENDED.xlsx',
 'Keyword_Creation/N11 - BANK TRANSFERS.xlsx',
 'Keyword_Creation/NX - ANTI CLASSIFICATION.xlsx',
 'Keyword_Creation/N17 - PENDING JUDGE DECISION.xlsx',
 'Keyword_Creation/N9 - PLACE OF RESIDENCY REQUIRED.xlsx',
 'Keyword_Creation/N6 - REJECTED TRANSFER OF LEGAL REPRE

## Categorise Files

In [4]:
ls1, ls3, s1 = [], [], []
for f in keyword_files:
    xl_file = pd.ExcelFile(f)
    dfs = xl_file.parse(xl_file.sheet_names[0])
    tic=False
    for index, row in dfs.iterrows():
        ite=dict()
        if(row[0]=='TICKETS' or row[0]=='TICKET'):
            tic=True
        if(row[0]!='TICKETS' and row[0]!='TICKET' and isinstance(row[0], basestring)):
        
            ite['file_class']=f.split('/')[-1].split()[0]
            s1.append(ite['file_class'])
            ite['notification_type']=row[0]
            if(tic):
                ite['file_type']='TICKET'
            else:
                ite['file_type']='NOTIFICATION'
            kw=list()
            for i in range(1,row.size):
                if(isinstance(row[i], basestring) and row[i]!='+'):
                    kw.append(unidecode.unidecode(row[i]).lower() )
            ite['keyword']=kw
            if(ite['file_class'] == 'NX'):
                ls3.append(ite)
            else:
                ls1.append(ite)
newkdf=pd.DataFrame(ls1)
suspkdf=pd.DataFrame(ls3)
suspkdf=suspkdf.rename(columns={"notification_type":"remove_class"})

In [7]:
newkdf.loc[newkdf['file_class']=='N17'][:10]

,file_class,file_type,keyword,notification_type
900,N17,NOTIFICATION,[dese cuenta a s.sa],N17 - PENDING JUDGE DECISION
901,N17,NOTIFICATION,[se da cuenta a s.sa],N17 - PENDING JUDGE DECISION
902,N17,NOTIFICATION,[dar cuenta a s.sa],N17 - PENDING JUDGE DECISION
903,N17,NOTIFICATION,[acuerdo dar cuenta a s.sa],N17 - PENDING JUDGE DECISION
904,N17,NOTIFICATION,[dese cuenta a la magistrado juez],N17 - PENDING JUDGE DECISION
905,N17,NOTIFICATION,[queden las actuaciones para resolver dicho re...,N17 - PENDING JUDGE DECISION
906,N17,NOTIFICATION,[pasen las actuaciones a la mesa de ssa],N17 - PENDING JUDGE DECISION
907,N17,NOTIFICATION,[dese cuenta a ssria],N17 - PENDING JUDGE DECISION
908,N17,NOTIFICATION,[pasen los autos a la],N17 - PENDING JUDGE DECISION
909,N17,NOTIFICATION,[mesa de ssa],N17 - PENDING JUDGE DECISION


## Specify Purpose

In [9]:
pdfclass={ 'N1':'CLAIM ADMITTED TO PROCEDURE',
           'N2':'ACCEPTED TRANSFER OF LEGAL REPRESENTATION',
           'N3':'ENFORCEMENT ORDER',
           'N4':'STATEMENT OF PAYMENT',
           'N5':'REJECTED CLAIMS',
           'N6':'REJECTED TRANSFER OF LEGAL REPRESENTATION',
           'N7':'HEARING',
           'N8':'ASSET INQUIRY',
           'N9':'PLACE OF RESIDENCY REQUIRED',
           'N10':'REQUIREMENT',
           'N11':'BANK TRANSFERS',
           'N12':'SOLICITOR REMOVAL',
           'N13':'SUSPENDED HEARINGS',
           'N14':'CANCELLED HEARINGS',
           'N15':'NEGATIVE ASSET INQUIRY',
           'N16':'PENDING ASSET INQUIRY',
           'N17' :'PENDING JUDGE DECISION',
           'N24' : 'FILED',
           'N34':'CASE SUSPENDED ON LACK OF LEGAL GROUNDS',
           'N35':'PROCEDURE SUSPENDED',
           'N36' : 'SUSPENSION LIFTED',
           'N19' : 'FURTHER GARNISHMENT',
           'N18' : 'GARNISHMENT',
           'N27' : 'REQUERIMIENTO CONCURSAL'
          }

newkdf['purpose']=''
n=newkdf.iterrows()
for i, row in n:
    if(not ( row['file_class'] =='N5'or row['file_class'] =='N6')):
        if(pdfclass[row['file_class']] in row['notification_type']):
            newkdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            newkdf.loc[i,'purpose']='EXTRACTION'
    elif row['file_class'] =='N5':
        if('N5 - REJECTED CLAIMS' in row['notification_type']):
            newkdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            newkdf.loc[i,'purpose']='EXTRACTION'
    elif row['file_class'] =='N6':
        if('N6 - REJECTED TRANSFER OF LEGAL REPRESENTATION' in row['notification_type']):
            newkdf.loc[i,'purpose']='CLASSIFICATION'
        else:
            newkdf.loc[i,'purpose']='EXTRACTION'
            
k=newkdf[(newkdf['file_class']=='N10') & (newkdf['purpose']=='CLASSIFICATION')&(newkdf['file_type']=='NOTIFICATION')].groupby('notification_type')
new10df=pd.DataFrame(columns=newkdf.columns)
i=0
for lidx in k.groups['N10 - REQUIREMENTS']:
    for ridx in k.groups['N10 - REQUIREMENT SPECIFICATION']:
        new10df=new10df.append(newkdf.iloc[lidx],ignore_index=True)
        
        new10df.loc[i,'keyword']=new10df.loc[i,'keyword']+newkdf.loc[ridx,'keyword']
        i=i+1
newkdf = newkdf[~((newkdf['file_class']=='N10') & (newkdf['purpose']=='CLASSIFICATION')&(newkdf['file_type']=='NOTIFICATION')&
                  ((newkdf['notification_type']=='N10 - REQUIREMENTS')|(newkdf['notification_type']=='N10 - REQUIREMENT SPECIFICATION')))].append(new10df,ignore_index=True)


In [11]:
#newkdf.loc[newkdf['file_class']=='N17']

In [12]:
k=newkdf[(newkdf['file_class']=='N27') & (newkdf['purpose']=='CLASSIFICATION')&(newkdf['file_type']=='NOTIFICATION')].groupby('notification_type')
new27df=pd.DataFrame(columns=newkdf.columns)
i=0
for lidx in k.groups['N27 - REQUERIMIENTO CONCURSAL Com2']:
    for ridx in k.groups['N27 - REQUERIMIENTO CONCURSAL Com3']:
        new27df=new27df.append(newkdf.iloc[lidx],ignore_index=True)
        
        new27df.loc[i,'keyword']=new27df.loc[i,'keyword']+newkdf.loc[ridx,'keyword']
        i=i+1
newkdf = newkdf[~((newkdf['file_class']=='N27') & (newkdf['purpose']=='CLASSIFICATION')&(newkdf['file_type']=='NOTIFICATION')&
                  ((newkdf['notification_type']=='N27 - REQUERIMIENTO CONCURSAL Com2')|(newkdf['notification_type']=='N27 - REQUERIMIENTO CONCURSAL Com3')))].append(new27df,ignore_index=True)

newkdf = newkdf.reset_index()
for i, r in newkdf.loc[(newkdf['file_class']=='N27') & ((newkdf['notification_type']=='N27 - REQUERIMIENTO CONCURSAL Com1')|(newkdf['notification_type']=='N27 - REQUERIMIENTO CONCURSAL Com2'))].iterrows():
    x= newkdf.loc[i,'keyword']+['administracion concursal']
    newkdf.set_value(i,'keyword',x)#]=list(x)
    newkdf.loc[i,'notification_type']= 'N27 - REQUERIMIENTO CONCURSAL'
    
newkdf = newkdf.rename(columns={'file_class':'fileclass','file_type':'filetype',"notification_type":"decision_type"})

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


In [17]:
#newkdf.loc[newkdf['fileclass']=='N7']

## Add Sub

In [18]:
warnings.simplefilter('ignore')
kdf=newkdf
kdf['keywordHash'] = kdf['keyword'].apply(lambda x :hashlib.sha256(json.dumps(x).encode('UTF-8')).hexdigest())
kdf['sub'] = kdf['keyword'].apply(lambda x :[])
kdf['bias']=kdf['fileclass']
kdf.loc[(kdf['fileclass']=='N2')&(kdf['filetype']=='NOTIFICATION'),'bias']='N12'

In [21]:

def add_sub_link_keyword(kdf,keywords,fileclass,filetype,purpose,decision_type,bias,parent_class):
    n_ind = kdf.index[-1]+1
    for n15k in keywords:
        k = [unidecode.unidecode(unicode(n15k,'UTF-8')).lower()]
        hsh=hashlib.sha256(json.dumps(k)).hexdigest()
         
#         if 'DE DICHA CONSULTA RESULTÓ' this add 'QUE NO PERTENECE A ESTE PARTIDO JUDICIAL'    
        
        if not hsh in kdf['keywordHash'].values.tolist():
            kdf.set_value(n_ind,'fileclass',fileclass)
            kdf.set_value(n_ind,'filetype',filetype)
            kdf.set_value(n_ind,'purpose',purpose)
            kdf.set_value(n_ind,'decision_type',decision_type)

            kdf.set_value(n_ind,'keyword',k)
            kdf.set_value(n_ind,'bias',bias)
            kdf.set_value(n_ind,'keywordHash',hsh)
            kdf.set_value(n_ind,'sub',[])
        for i in kdf[(kdf['fileclass']==parent_class)&(kdf['filetype']=='NOTIFICATION')&(kdf['purpose']=='CLASSIFICATION')].index:
            
            kx = kdf[kdf.index==i]['sub'].values[0]
            kx.append(hsh)
            kdf.set_value(i,'sub',kx)
        n_ind+=1
    return kdf
N14_from_N13 = ['DESISTIMIENTO DEL PROCESO', 'HABIENDOSE SOLICITADO DESISTIMIENTO', 'HABER LLEGADO A UN ACUERDO',
                'ACUERDO EL ARCHIVO DEFINITIVO DE LAS ACTUACIONES', 'ARCHIVAR EL PRESENTE PROCEDIMIENTO',
                'ARCHIVAR LAS ACTUACIONES', 'ARCHÍVASE EL EXPEDIENTE', 'ARCHÍVENSE LOS PRESENTES AUTOS',
                'DANDO PUBLICIDAD A LA CONCLUSIÓN Y ARCHIVO', 'DAR POR TERMINADO EL PRESENTE EXPEDIENTE',
                'DAR POR TERMINADO EL PRESENTE PROCEDIMIENTO', 'DE DICHA CONSULTA RESULTÓ', 
                'DECLARAR TERMINADO EL PRESENTE PROCEDIMIENTO DE EJECUCIÓN',
                'DECLARO FINALIZADO EL PRESENTE PROCEDIMIENTO MONITORIO',
                'SE ACUERDA DAR POR TERMINADO EL PROCESO MONITORIO',
                'SE ACUERDA EL ARCHIVO DE LAS ACTUACIONES', 'SE ACUERDA EL ARCHIVO DEFINITIVO',
                'SE DECLARA FINALIZADO EL PRESENTE PROCESO MONITORIO', 'SE DECLARA TERMINADO',
                'SE DECRETA LA TERMINACION DEL PROCESO', 'TENER POR TERMINADO']

N13_from_N7 = ['SUSPENDO','SE SUSPENDE','SUSPENSIÓN','SUSPENDIÉNDOSE','SUSPENDER','ACORDANDO LA SUSPENSIÓN', 
               'SE ACUERDA NUEVO SEÑALAMIENTO','SEÑALÁNDOSE NUEVAMENTE','SEÑALAMIENTO NUEVO','SE SEÑALA NUEVAMENTE']
#Parent->N2,Notification,classification
N2_solicitor_list=["MALAGON LOYO","ALCOCER ANTON","GARCIA ABASCAL","GARCÍA ABASCAL"]
#Parent->N4,Notification,classification
N6_from_N2=["NO HA LUGAR A TENER POR PERSONADA","NO HA LUGAR A LO SOLICITADO"]
N2_from_N6=["EN ESTE CASO HABIÉNDOSE JUSTIFICADO DE MODO FEHACIENTE LA REALIDAD DE LA CESIÓN"]
N9_from_N10=["APORTE NOTA SIMPLE DEL REGISTRO MERCANTIL","ES DESCONOCIDO EN ESTE DOMICILIO"]
N11_from_N4 = ['TRANSFERENCIA','EL No DE CUENTA DESIGNADO','A LA CUENTA BANCARIA DESIGNADA',
               'MEDIANTE TRANSFERENCIA A LA CUENTA DESIGNADA','MEDIANTE TRANSFERENCIA', 'SERA TRANSFERIDO']
N3_from_N10 = ['NOTIFIQUESE ESTA RESOLUCION AL/LOS EJECUTADOS CON ENTREGA DE COPIA DE LA DEMANDA EJECUTIVA']
#Parent->N16,Notification,classification
N10_from_N11=['REQUIÉRASE A LA PARTE EJECUTANTE A FIN DE QUE DESIGNE CUENTA BANCARIA']
N5_to_N10=['BAJO APERCIBIMIENTO DE QUE DE NO VERIFICARSE','SE ACUERDA LA PRÓRROGA DEL PLAZO','SE PROCEDERÁ A LA INADMISIÓN A TRÁMITE']
N4_from_N11=['CUENTA DE FONDOS PROVISIONALMENTE ABANDONADOS','MANDAMIENTO DE PAGO',
             'MANDAMIENTO DE DEVOLUCIÓN','MANDAMIENTO TELEMÁTICO DE PAGO', 'COPIA DE TRANSFERENCIA']
N13_from_N14 = ['HABIÉNDOSE SOLICITADO POR LA PARTE DEMANDANTE EL DESISTIMIENTO DEL PROCESO', 
                'ACORDANDOSE LO QUE PROCEDA UNA VEZ TRASCURRA EL PLAZO']
N19_from_N18 = ['MEJORA DEL EMBARGO']

kdf = add_sub_link_keyword(kdf,N19_from_N18,'N19','NOTIFICATION','CLASSIFICATION_2','N19 - FURTHER GARNISHMENT','N19','N18')

kdf = add_sub_link_keyword(kdf,N13_from_N7,'N13','NOTIFICATION','CLASSIFICATION_2','N13 EXTRACTION','N13','N7')
kdf = add_sub_link_keyword(kdf,N14_from_N13,'N14','NOTIFICATION','CLASSIFICATION_2','N14-CANCELLED HEARINGS','N14','N13')
kdf = add_sub_link_keyword(kdf,N6_from_N2,'N6','NOTIFICATION','CLASSIFICATION_2','N6-REJECTED TRANSFER OF LEGAL REPRESENTATION','N6','N2')
kdf = add_sub_link_keyword(kdf,N2_from_N6,'N2','NOTIFICATION','CLASSIFICATION_2','N2- TRANSFER OF LEGAL REPRESENTATION','N2','N6')
kdf = add_sub_link_keyword(kdf,N9_from_N10,'N9','NOTIFICATION','CLASSIFICATION_2','N9- PLACE OF RESIDENCY REQUIRED','N9','N10')

kdf = add_sub_link_keyword(kdf,N2_solicitor_list,'N2','NOTIFICATION','AUXILIARY','N2-SOLICITOR LIST','N2','N2')
kdf = add_sub_link_keyword(kdf,N11_from_N4,'N11','NOTIFICATION','CLASSIFICATION_2','N11-BANK TRANSFERS','N11','N4')
kdf = add_sub_link_keyword(kdf,N5_to_N10,'N10','NOTIFICATION','CLASSIFICATION_2','N10-REQUIREMENTS','N10','N5')
kdf = add_sub_link_keyword(kdf,N4_from_N11,'N4','NOTIFICATION','CLASSIFICATION_2','N4-STATEMENT OF PAYMENT','N4','N11')
kdf = add_sub_link_keyword(kdf,N10_from_N11,'N10','NOTIFICATION','CLASSIFICATION_2','N10-REQUIREMENTS','N10','N11')
kdf = add_sub_link_keyword(kdf, N3_from_N10, 'N3', 'NOTIFICATION', 'CLASSIFICATION_2', 'N3 EXTRACTION', 'N3', 'N10')
kdf = add_sub_link_keyword(kdf, N13_from_N14, 'N13', 'NOTIFICATION', 'CLASSIFICATION_2', 'N13 EXTRACTION', 'N13', 'N14')

In [ ]:
#kdf

## Maintain Classification Hierarchy  

In [ ]:
for k1 in N6_from_N2:
    k = [unidecode.unidecode(unicode(k1,'UTF-8')).lower()]
    hsh=hashlib.sha256(json.dumps(k)).hexdigest()
    for i in kdf[(kdf['fileclass']=='N2')&(kdf['purpose']=='AUXILIARY')].index:
            kx = kdf[kdf.index==i]['sub'].values[0]
            kx.append(hsh)
            kdf.set_value(i,'sub',kx)

keys = [[unidecode.unidecode(unicode('RESUMEN DEL RESULTADO','UTF-8')).lower(),
         unidecode.unidecode(unicode('CONSULTA INTEGRAL','UTF-8')).lower(),
         unidecode.unidecode(unicode('RESUMEN DE INCIDENCIAS O INFORMACIÓN NO ENCONTRADA ','UTF-8')).lower()
        ],
        [
         unidecode.unidecode(unicode('CONSULTA DE SITUACIONES LABORALES DE LA TGSS','UTF-8')).lower(),
         unidecode.unidecode(unicode('SITUACION ACTUAL: ALTA','UTF-8')).lower()],
        [
         unidecode.unidecode(unicode('CONSULTA DE SITUACIONES LABORALES DE LA TGSS','UTF-8')).lower(),
         unidecode.unidecode(unicode('ALTA SITUACION ACTUAL','UTF-8')).lower()],
        [
         unidecode.unidecode(unicode('CONSULTA DE VIDA LABORAL DE LA TGSS','UTF-8')).lower(),
         unidecode.unidecode(unicode('RÉGIMEN DE LA SS: GENERAL','UTF-8')).lower()
        ],
        [
         unidecode.unidecode(unicode('CONSULTA DE VIDA LABORAL DE LA TGSS','UTF-8')).lower(),
         unidecode.unidecode(unicode('GENERAL RÉGIMEN DE LA SS:','UTF-8')).lower()
        ]
       ]
         
for k in keys:
    hsh8=hashlib.sha256(json.dumps(k)).hexdigest()
    n_ind = kdf.index[-1]+1
    if not hsh8 in kdf['keywordHash'].values.tolist():
        kdf.set_value(n_ind,'fileclass','N8')
        kdf.set_value(n_ind,'filetype','NOTIFICATION')
        kdf.set_value(n_ind,'purpose','CLASSIFICATION')
        kdf.set_value(n_ind,'decision_type','N8-ASSET INQUIRY')

        kdf.set_value(n_ind,'keyword',k)
        kdf.set_value(n_ind,'bias','N8')
        kdf.set_value(n_ind,'keywordHash',hsh8)
        kdf.set_value(n_ind,'sub',[])
    for i in kdf[(kdf['fileclass']=='N16')&(kdf['purpose']=='CLASSIFICATION')].index:

        kx = kdf[kdf.index==i]['sub'].values[0]
        kx.append(hsh8)
        kdf.set_value(i,'sub',kx)
keys = [[unidecode.unidecode(unicode('CONSULTA INTEGRAL','UTF-8')).lower(),
         unidecode.unidecode(unicode('RESUMEN DE INCIDENCIAS O INFORMACION NO ENCONTRADA','UTF-8')).lower()],
        [unidecode.unidecode(unicode('CONSULTA DE SITUACIONES LABORALES DE LA TGSS','UTF-8')).lower(),
         unidecode.unidecode(unicode('SITUACION ACTUAL: BAJA','UTF-8')).lower()],
        [unidecode.unidecode(unicode('CONSULTA DE VIDA LABORAL DE LA TGSS','UTF-8')).lower(),
         unidecode.unidecode(unicode('AFILIADO SIN SITUACIONES EN EL PERIODO','UTF-8')).lower()],
       ]
for k in keys:
    hsh=hashlib.sha256(json.dumps(k)).hexdigest()
    n_ind = kdf.index[-1]+1
    if not hsh in kdf['keywordHash'].values.tolist():
        kdf.set_value(n_ind,'fileclass','N15')
        kdf.set_value(n_ind,'filetype','NOTIFICATION')
        kdf.set_value(n_ind,'purpose','CLASSIFICATION_2')
        kdf.set_value(n_ind,'decision_type','N15-NEGATIVE ASSET INQUIRY')

        kdf.set_value(n_ind,'keyword',k)
        kdf.set_value(n_ind,'bias','N15')
        kdf.set_value(n_ind,'keywordHash',hsh)
        kdf.set_value(n_ind,'sub',[hsh8])
    for i in kdf[(kdf['fileclass']=='N16')&(kdf['filetype']=='NOTIFICATION')&(kdf['purpose']=='CLASSIFICATION')].index:

        kx = kdf[kdf.index==i]['sub'].values[0]
        kx.append(hsh8)
        kdf.set_value(i,'sub',kx)
        
        
        


## Remove Duplicates

In [23]:
kdf['keyword'] = kdf['keyword'].apply(lambda x : json.dumps(x))
kg=kdf[kdf['purpose']=='CLASSIFICATION'].groupby('keyword')
ls=[]
for k,v in kg.groups.items():
    if(len(v)>1):
        if len(set(kdf[kdf['keyword']==k]['fileclass'].values)) ==1:
            ls=ls+v.values.tolist()[1:]
        else:
            if set(kdf[kdf['keyword']==k]['fileclass'].values)<=set(['N8','N15','N16']):
                for j in v.values.tolist():
                    if kdf.loc[j,'fileclass']!='N16':
                        ls.append(j)
            elif set(kdf[kdf['keyword']==k]['fileclass'].values)<=set(['N13','N14']):
                for j in v.values.tolist():
                    if kdf.loc[j,'fileclass']!='N13':
                        ls.append(j)
print(ls)
kdf['keyword'] = kdf['keyword'].apply(lambda x : json.loads(x))

kdf=kdf.drop(kdf.index[ls])

[388, 374, 631, 15, 642, 360, 86, 87, 415, 531, 401, 640, 639, 633, 88, 2273, 632, 753, 233, 234, 822, 232, 672, 668, 126, 634, 727, 641, 457, 638, 443]


## Write Files

In [24]:
kdf.shape

(4165, 9)

In [25]:
a = {'keywords':kdf,
     'susKeyword':suspkdf
    }
with open('Keyword.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
kdf.shape, suspkdf.shape

((4165, 9), (338, 4))

In [ ]:
kdf.loc[(kdf.fileclass == 'N27')]